In [0]:
dbutils.secrets.help() 


Provides utilities for leveraging secrets within notebooks.
Databricks documentation for more info.
 get(scope: String, key: String): String -> Gets the string representation of a secret value with scope and key getBytes(scope: String, key: String): byte[] -> Gets the bytes representation of a secret value with scope and key list(scope: String): Seq -> Lists secret metadata for secrets within a scope listScopes: Seq -> Lists secret scopes

Below we list the secrets that we have created and saved in the vault and then allowed databricks to find them

In [0]:
dbutils.secrets.list("vault-scope")


Out[6]: [SecretMetadata(key='databricks-app-client-id'),
 SecretMetadata(key='databricks-app-client-secret'),
 SecretMetadata(key='databricks-app-tenant-id')]

As you see below, this makes the secrets not readily available to the public, say on github where this notebook has been publicly made available.

In [0]:
dbutils.secrets.get(scope="vault-scope",key="databricks-app-client-secret")

Out[15]: '[REDACTED]'

Reading the client and tenant ids and the client secret

In [0]:
storage_account_name="1stdatalake"
client_id=dbutils.secrets.get(scope="vault-scope",key="databricks-app-client-id")
tenant_id=dbutils.secrets.get(scope="vault-scope",key="databricks-app-tenant-id")
client_secret=dbutils.secrets.get(scope="vault-scope",key="databricks-app-client-secret")


In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": f"{client_id}",
           "fs.azure.account.oauth2.client.secret": f"{client_secret}",
           "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
         

Mounting the container in the ADLS; this is a pointer to the container that allows a connection and exchange of files between the created blob containers and databricks notebook, we have to use it to import the .csv into the databricks. A function allows us to repeat this process multiple time effortlessly by just

In [0]:

def mount_adls(container_name):
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account_name}/{container_name}",
        extra_configs = configs)

calling the function, argument is the container name

In [0]:
mount_adls("data")

In [0]:
mount_adls("results")

In [0]:
dbutils.fs.ls("/mnt/1stdatalake/data")
dbutils.fs.ls("/mnt/1stdatalake/results")

Out[22]: []